# **Sensitivity of Bond Prices**

In [2]:
from datetime import datetime
from datetime import timedelta
import scipy.optimize as optimize

## Compute the price of a 9% semiannual coupon bond with 10 years to maturity and a face value of USD 1,000. Assume that the market required yield (YTM) is 6%.

In [3]:
def bond_price(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq=1):
  ytm = ytm_p/100
  freq = float(freq)
  periods = years_to_maturity*freq
  coupon = (coupon_rate_p/100)*face_value/freq
  dt = [(i+1)/freq for i in range(int(periods))]
  price = sum([coupon/(1+ytm/freq)**(freq*t) for t in dt]) + face_value/(1+ytm/freq)**(freq*years_to_maturity)
  
  return round(price,2)

face_value = 1000
coupon_rate_p = 9
freq = 2
ytm_p = 6
years_to_maturity = 10

price = bond_price(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq)
print(f'Price: ${price}')

Price: $1223.16


## Compute the price of a 9% annual coupon bond with 10 years to maturity and a face value of USD 1,000. Assume that the market required yield (YTM) is 6%.

In [4]:
face_value = 1000
coupon_rate_p = 9
freq = 1
ytm_p = 6
years_to_maturity = 10

price = bond_price(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq)
print(f'Price: ${price}')

Price: $1220.8


## Compute the effective YTM of a 7% semiannual coupon bond with 15 years to maturity and face value of USD 1,000. The market bond price is USD 655.9.


*   Assuming that the annual YTM is 10%
*   Assuming that the annual YTM is 12%



In [5]:
def bond_price2(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq=1):
  ytm = ytm_p/100
  freq = float(freq)
  periods = years_to_maturity*freq
  coupon = (coupon_rate_p/100)*face_value/freq
  price = (coupon*freq/ytm)*(1 - 1/(1 + ytm/freq)**periods) + face_value/(1 + ytm/freq)**periods
  
  return round(price,1)

face_value = 1000
coupon_rate_p = 7
freq = 2
years_to_maturity = 15
market_price = 655.9

ytm = 10
price10 = bond_price2(face_value, years_to_maturity, ytm, coupon_rate_p, freq)
print(f'Price10: ${price10} > {market_price}, {ytm}% cannot be the YTM')

ytm = 12
price12 = bond_price2(face_value, years_to_maturity, ytm, coupon_rate_p, freq)
print(f'Price12: ${price12} = {market_price}')

effective_ytm = ((1 + (ytm/100)/freq)**freq) - 1
print(f'Effective yield: {round(100*effective_ytm, 2)}%')

Price10: $769.4 > 655.9, 10% cannot be the YTM
Price12: $655.9 = 655.9
Effective yield: 12.36%


## Compute the YTM of a 8% semiannual coupon bond with 20 years to maturity and a face value of USD 1,000. Assume that the market price of the bond is 900. Find the semiannual, annual and effective annual YTM for that bond. 

In [6]:
def bond_ytm(price, face_value, maturity_years, interest_rate_percent, freq=1, ytm=0.05):
    freq = float(freq)
    periods = maturity_years*freq
    coupon = face_value * (interest_rate_percent/100)/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y : \
    sum([coupon / (1 + y / freq)**(freq * t) for t in dt]) + \
    face_value / (1 + y / freq)**(freq * max(dt)) - price

    return round(100*optimize.newton(ytm_func, ytm), 2)

face_value = 1000
coupon_rate_p = 8
years_to_maturity = 20
market_price = 900

freq = 2
ytm_semi = bond_ytm(market_price, face_value, years_to_maturity, coupon_rate_p, freq)/2
print(f'Semiannual YTM: {ytm_semi}%')

ytm = ytm_semi*freq
print(f'Annual YTM: {ytm}%')

effective_ytm = (1 + (ytm_semi/100))**freq - 1
print(f'Effective YTM: {round(100*effective_ytm, 2)}%')

Semiannual YTM: 4.545%
Annual YTM: 9.09%
Effective YTM: 9.3%


## Compute the YTM of a 8% annual coupon bond with 20 years to maturity and a face value of USD 1,000. Assume that the market price of the bond is 900.

In [7]:
face_value = 1000
coupon_rate_p = 8
years_to_maturity = 20
market_price = 900
freq = 1

ytm = bond_ytm(market_price, face_value, years_to_maturity, coupon_rate_p, freq)
print(f'YTM: {ytm}%')

YTM: 9.1%


## Find the semiannual, annual and effective annual for that bond yield-to-call (YTC) of a 10% semiannual coupon bond with 20 years to maturity, a face value of USD 1,000 and price of callable bond in 6 years of USD 1,025. Assume that the market price of the bond is 848. 



In [47]:
def bond_ytc(price, face_value, price_callable, years_to_call, interest_rate_percent, freq=1, ytm=0.05):
    freq = float(freq)
    periods = years_to_call*freq
    coupon = face_value * (interest_rate_percent/100)/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y : \
    sum([coupon / (1 + y / freq)**(freq * t) for t in dt]) + \
    price_callable / (1 + y / freq)**(freq * max(dt)) - price

    return round(100*optimize.newton(ytm_func, ytm), 2)

def ytc_price(fv, pc, crp, ytcs, ytm, freq=1):
  ytcs = (ytcs/100)
  periods = ytm*freq
  coupon = (crp/100)*fv/freq
  f1 = coupon/ytcs
  factor = (1 + ytcs)**periods
  price = f1*(1 - 1/factor) + pc/factor
  
  return round(price, 2)

face_value = 1000
coupon_rate_p = 10
years_to_maturity = 20
market_price = 848
years_to_call = 6
price_callable = 1025

freq = 2
ytc_semi = bond_ytc(market_price, face_value, price_callable, years_to_call, coupon_rate_p, freq)/2
print(f'Semiannual YTC: {ytc_semi}%')

ytc = ytc_semi*freq
print(f'Annual YTC: {ytc}%')

effective_ytc = (1 + (ytc_semi/100))**freq - 1
print(f'Effective YTC: {round(100*effective_ytc, 2)}%')

pv = ytc_price(face_value, price_callable, coupon_rate_p, ytc_semi, years_to_call, freq)
print(f'Price value: ${pv}')

gap = round(pv - market_price, 2)
gapp = round(pv / market_price, 2)
print(f'Gap: ${gap} {gapp}%')

Semiannual YTC: 7.06%
Annual YTC: 14.12%
Effective YTC: 14.62%
Price value: $847.93
Gap: $-0.07 1.0%


## Find the semiannual, annual and zero coupon for a bond of a 9% coupon rate with 20 years to maturity, a face value of USD 1,000. Assume that the semiannual YTM is 6%.

In [61]:
def bond_price(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq=1):
  ytm = ytm_p/100
  freq = float(freq)
  periods = years_to_maturity*freq
  coupon = (coupon_rate_p/100)*face_value/freq
  dt = [(i+1)/freq for i in range(int(periods))]
  price = sum([coupon/(1+ytm/freq)**(freq*t) for t in dt]) + face_value/(1+ytm/freq)**(freq*years_to_maturity)
  
  return round(price,2)

face_value = 1000
coupon_rate_p = 9
ytm_p = 6
years_to_maturity = 20

freq = 2
price = bond_price(face_value, years_to_maturity, ytm_p, coupon_rate_p, freq)
print(f'Price semiannual frequency: ${price}')

effective_ytm = round(100*(((1 + (ytm_p/100)/freq)**freq) - 1), 2)

freq = 1
price = bond_price(face_value, years_to_maturity, effective_ytm, coupon_rate_p, freq)
print(f'Price annual frequency: ${price}')

zero = bond_price(1000, 20, 6, 0, 2)
print(f'Zero-coupon bond: ${zero}')

Price semiannual frequency: $1346.72
Price annual frequency: $1331.35
Zero-coupon bond: $306.56


## Find the bond price of a 9% semiannual coupon rate with 20 years to maturity, a maturity value of USD 1,000. Assume that the semiannual YTMs are 6 - 10%.

In [64]:
face_value = 1000
coupon_rate_p = 9
years_to_maturity = 20
freq = 2

ytm_p = [6, 7, 8, 9 , 10]

for p in ytm_p:
  price = bond_price(face_value, years_to_maturity, p, coupon_rate_p, freq)
  if price > face_value:
    kind = "at a premium"
  elif price < face_value:
    kind = "at a discount"
  else:
    kind = "at par"
  print(f'Price ytm = {p}%: ${price} {kind}')

Price ytm = 6%: $1346.72 at a premium
Price ytm = 7%: $1213.55 at a premium
Price ytm = 8%: $1098.96 at a premium
Price ytm = 9%: $1000.0 at par
Price ytm = 10%: $914.2 at a discount


## Find the bond price of a 8% semiannual coupon rate with 20 years to maturity, a maturity value of USD 1,000. The market required yield are: are 6, 8 and 10%.